<h2> Lab 9

<h3> Exercise Set 1

In [2]:
import gymnasium as gym
import numpy as np
import tensorflow as tf #you can tf.keras to access most of the needed items
from collections import deque #get to know this package, it is amazing to build your replay buffer :)
import random

ModuleNotFoundError: No module named 'gymnasium'

In [3]:
# Hyperparameters
learning_rate = 0.001 #use mse loss, adam optimizer, relu and linear activations
gamma = 0.99 #reward discount factor
epsilon_init = 1.0 #start with full exploration
epsilon_min = 0.01 #towards the end you are only 10\% likely to explore, otherwise exploit
epsilon_decay = 0.995 #decay after every episode
batch_size = 64 #for gradient descent
memory_size = 2000 #size of the replay buffer
target_update_freq = 10 #parameter C in slides, when to copy the primary to the target network
num_episodes = 500 #total episodes for training
num_layers = [64,64] #these are hidden layers only for the Q/primary network
num_epochs = 3 #only fit the primary/Q network for 3 epochs to finish fast

# Create the environment (for testing )
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0] #this gives you indication of the state space
action_size = env.action_space.n #this gives you indication of the action space

NameError: name 'gym' is not defined

In [3]:
# 1. Initialize replay memory (ReplayBuffer) to store past experiences
replayBuffer = deque()

# 2. Initialize Q−network (a.k.a Primary Network) with random weights
primaryQN = tf.keras.Sequential()
primaryQN.add(tf.keras.Input(shape=(4,)))
primaryQN.add(tf.keras.layers.Dense(50, activation='relu'))
primaryQN.add(tf.keras.layers.Dense(50, activation='relu'))
primaryQN.add(tf.keras.layers.Dense(2, activation='relu'))

# 3. Initialize target Q−network (Target−Q−network) with same weights as Q−network
targetQN = tf.keras.Sequential()
targetQN.add(tf.keras.Input(shape=(4,)))
targetQN.add(tf.keras.layers.Dense(50, activation='relu'))
targetQN.add(tf.keras.layers.Dense(50, activation='relu'))
targetQN.add(tf.keras.layers.Dense(2, activation='relu'))
targetQN.set_weights(primaryQN.get_weights())

In [ ]:
epsilon = epsilon_init

for i in range(num_episodes):
    state = env.reset()
    done = False
    for j in range(500):
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample()
        else:
            action = targetQN.predict(state)

        nextState, reward, done, info = env.step(action)
        replayBuffer.append([state, action, reward, nextState])

        # 14. If replay memory contains enough samples:
        if len(replayBuffer) >= batch_size:
            # 15. Sample a batch of transitions (state , action , reward, next state) from replay memory

            # 16. For each transition in the mini−batch: #this is the most tricky part!
                # 17. If next state is terminal:
                    # 18. Set target = reward
                # 19. Otherwise:
                    # 20. Set target = reward + discount factor ∗ max(Target−Q−network(next state))
                # 21. Perform gradient descent step to minimize loss :
                    # loss = (target − Q−network(state, action))ˆ2
                #22. Update Q−network weights using gradient descent
                
        state = nextState

        if j % target_update_freq == 0:
              targetQN.set_weights(primaryQN.get_weights())

        epsilon = epsilon * epsilon_decay



<h3> Exercise Set 2

In [4]:
#       0 1 2 3 4 5 6 7 8 9
#     0 - - - - - - - - - -
#     1 - - - - - - - - - -
#     2 - - - - - - - - - -
#     3 - - - - - - - - - -
#     4 - - - - - - - O - -
#     5 - - - - - - - - - -
#     6 - - - - - - - - - -
#     7 - - - - O - - - - -
#     8 - - - - O - - - - -
#     9 - - - - - - - - - G

    # To translate continuous variables:
    # 0.0-0.9 = 0
    # 1.0-1.9 = 1
    # ...
    # 9.0-10.0 = 9
    # Goal is at (10,10) exactly


class CustomGridEnv(gym.Env):
    def __init__(self):
        super(CustomGridEnv, self).__init__()
        # Actions on a 2d cartesian grid
        self.action_space = gym.spaces.Box(low=-1.0, high=1.0, shape=(2,), \
                                           dtype=np.float32) 
        # Observe current position (x,y) and goal position (x,y) in 2d array
        self.observation_space = gym.spaces.Box(low=0, high=10.0, \
                                                shape = (2,2), dtype=np.float32)
        # Define the initial state: agent at (0,0), goal at (10,10)
        self.state = np.array([[0, 0], [10, 10]], dtype=np.float32)
        # Define the obstacles (fixed)
        self.obstacles = np.array([[4,7], [4,8], [7,4]])

    def reset(self):
        # reset to a random position
        self.state[0,:] = np.random.uniform(0, 10, size=2)
        # check for spawning inside obsstacles
        for i in range(self.obstacles.shape[0]):
            if np.floor(self.state[0,0]) == self.obstacles[i,0] and \
               np.floor(self.state[0,1]) == self.obstacles[i,1]:
                # if spawned inside an obstacle, shunt up and to the left
                self.state[0,0] = np.random.uniform(self.obstacles[i,0] - 1, \
                                                  self.obstacles[i,0])
                self.state[0,1] = np.random.uniform(self.obstacles[i,1] - 1, \
                                                  self.obstacles[i,1])
        return self.state
    
    def step(self, action):
        # check for valid action
        assert self.action_space.contains(action), "Invalid Action"
        done = False
        # take action
        self.state[0,0] += action[0]
        self.state[0,1] += action[1]
        self.state = np.clip(self.state, 0, 10)
        # reward based on distance
        reward = self.state[0,0] - self.state[1,0]
        reward += self.state[0,1] - self.state[1,1]
        # augment reward but do not end if obstacle was hit
        for i in range(self.obstacles.shape[0]):
            if np.floor(self.state[0,0]) == self.obstacles[i,0] and \
               np.floor(self.state[0,1]) == self.obstacles[i,1]:
                reward += -3
                done = False    # could choose to kill on obstacle collision instead
        # augment reward and end if goal was reached
        if np.floor(self.state[0,0]) == self.state[1,0] and \
           np.floor(self.state[0,1]) == self.state[1,1]:
            reward += 10
            done = True
        return self.state, reward, done
    

    def render(self):
        # round continuous values for rendering
        x = np.floor(self.state[0,0]) if self.state[0,0] != 10 else 9
        y = np.floor(self.state[0,1]) if self.state[0,1] != 10 else 9

        # If agent is in range [9,10],[9,10], no 'G' will print
        for i in range(10):
            print('')
            for j in range(10):
                if j == x and i == y:
                    print('X ', end='')  # agent pos
                elif j == 9 and i == 9:
                    print('G ', end='')  # goal
                elif j == self.obstacles[0,0] and i == self.obstacles[0,1]:
                    print('O ', end='')  # obstacle 1
                elif j == self.obstacles[1,0] and i == self.obstacles[1,1]:
                    print('O ', end='')  # obstacle 2
                elif j == self.obstacles[2,0] and i == self.obstacles[2,1]:
                    print('O ', end='')  # obstacle 3
                else:
                    print('- ', end='')  # nothing
                    
    def close(self):
        pass




In [5]:
# Example of using the environment
env = CustomGridEnv()
# Reset the environment to the initial state
obs = env.reset()
env.render()
print('')
print(f"Inital State: {env.state[0]}")
done = False
total_reward = 0
while not done:
    # Choose a random action (small random movement)
    action = env.action_space.sample()
    # Take a step in the environment
    state, reward, done = env.step(action)
    total_reward += reward
    # Render the current state of the environment
    env.render()
    print('')
    print(f"Action: {action}, State: {state[0,:]}, Reward: {reward}, Done: {done}")
print(f"Total Reward: {total_reward}")


- - - - - - - - - - 
- - - - - - - - - - 
- - - - - - - - - - 
- - - - - - - - - - 
- - - - - - - O - - 
- - - - - - - - - - 
- - - - - - X - - - 
- - - - O - - - - - 
- - - - O - - - - - 
- - - - - - - - - G 
Inital State: [6.1283712 6.7398863]

- - - - - - - - - - 
- - - - - - - - - - 
- - - - - - - - - - 
- - - - - - - - - - 
- - - - - - - O - - 
- - - - - - - - - - 
- - - - - - X - - - 
- - - - O - - - - - 
- - - - O - - - - - 
- - - - - - - - - G 
Action: [0.36334908 0.14131539], State: [6.49172   6.8812017], Reward: -6.627078056335449, Done: False

- - - - - - - - - - 
- - - - - - - - - - 
- - - - - - - - - - 
- - - - - - - - - - 
- - - - - - - O - - 
- - - - - - - - - - 
- - - - - - - - - - 
- - - - O X - - - - 
- - - - O - - - - - 
- - - - - - - - - G 
Action: [-0.54921275  0.8604576 ], State: [5.9425073 7.741659 ], Reward: -6.315833568572998, Done: False

- - - - - - - - - - 
- - - - - - - - - - 
- - - - - - - - - - 
- - - - - - - - - - 
- - - - - - - O - - 
- - - - - - - - -

<h3> Exercise Set 3

In [2]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

myEnv = make_vec_env("HalfCheetah-v4", n_envs=32)
model = PPO("MlpPolicy", myEnv, seed=42, verbose=1)
model.learn(total_timesteps=25000)
model.save("ppo_cartpole")
del model

ModuleNotFoundError: No module named 'stable_baselines3'